In [1]:
import pandas as pd
import numpy as np
import os
import importlib
import sys

In [2]:
backtesting_path = r'C:\Users\vinayak\Desktop\Backtesting'
print("path before", sys.path)
if backtesting_path not in sys.path:
    print("Backtesting was not in sys path")
    sys.path.append(backtesting_path)
print("path after", sys.path)
print()

path before ['c:\\Program Files\\Python312\\python312.zip', 'c:\\Program Files\\Python312\\DLLs', 'c:\\Program Files\\Python312\\Lib', 'c:\\Program Files\\Python312', '', 'C:\\Users\\vinayak\\AppData\\Roaming\\Python\\Python312\\site-packages', 'c:\\Program Files\\Python312\\Lib\\site-packages', 'c:\\Program Files\\Python312\\Lib\\site-packages\\win32', 'c:\\Program Files\\Python312\\Lib\\site-packages\\win32\\lib', 'c:\\Program Files\\Python312\\Lib\\site-packages\\Pythonwin']
Backtesting was not in sys path
path after ['c:\\Program Files\\Python312\\python312.zip', 'c:\\Program Files\\Python312\\DLLs', 'c:\\Program Files\\Python312\\Lib', 'c:\\Program Files\\Python312', '', 'C:\\Users\\vinayak\\AppData\\Roaming\\Python\\Python312\\site-packages', 'c:\\Program Files\\Python312\\Lib\\site-packages', 'c:\\Program Files\\Python312\\Lib\\site-packages\\win32', 'c:\\Program Files\\Python312\\Lib\\site-packages\\win32\\lib', 'c:\\Program Files\\Python312\\Lib\\site-packages\\Pythonwin', 'C:

In [3]:
# from scipy.optimize import minimize
import TradeAndLogics as TL
import Plot
import Data as data
import Data_Processing as dp
from enums import *

In [4]:
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA

In [5]:
def pwd():
    return os.getcwd()

In [6]:
class const:
    def __init__(self, value):
        self._value = value
    @property
    def value(self):
        return self._value
    @value.setter
    def value(self, value):
        return

In [7]:
pwd()

'c:\\Users\\vinayak\\Desktop\\Backtesting\\Dispersion\\PCA'

In [8]:
original_stdout = const(sys.stdout)
original_stdout.value

In [9]:
def fix_output(*file):
    for f in file:
        f.close()
    sys.stdout = original_stdout.value

In [10]:
def reload_modules():
    importlib.reload(TL)
    importlib.reload(Plot)
    importlib.reload(data)
    importlib.reload(dp)

In [11]:
fix_output()

In [12]:
reload_modules()

TRADING/ STRATEGY PARAMETERS

In [13]:
index_symbol = 'BANKNIFTY'
start_date = '2023-07-01'
end_date = '2023-12-01'
expiry_type = 'I'
risk_free_rate = 0.1 # (10% interest rate)
timeframe = 15 # mins
look_back_window = 5*25*4
# Trade/ Strategy Parameters
index_delta_threshold_per_lot = 5
component_delta_threshold_per_lot = 1
notional_vega = 10000 # rs
profit_target = 50000
stop_loss = 50000
zscore_threshold_long = 1.5
zscore_threshold_short = -1
epsilon = 0.2

In [14]:
strategy_desc = f"Dispersion_PCA_{index_symbol}"

In [15]:
def round(x, dig=2):
    return np.round(x, dig)

In [16]:
def insert_index_component(constituents, constituent_symbol, weight, lot_size):
    dict = {
        'weight': weight,
        'lot_size': lot_size
    }
    constituents[constituent_symbol] = dict

In [17]:
constituents_weight_lots = {}

In [18]:
index_symbol = 'BANKNIFTY'
index_lot_size = 15

In [19]:
insert_index_component(constituents_weight_lots, "PNB", 0.91, 8000)
insert_index_component(constituents_weight_lots, "ICICIBANK", 23.03, 700)
insert_index_component(constituents_weight_lots, "AUBANK", 2.69, 1000)
insert_index_component(constituents_weight_lots, "BANKBARODA", 1.84, 2925)
insert_index_component(constituents_weight_lots, "FEDERALBNK", 1.68, 5000)
insert_index_component(constituents_weight_lots, "IDFCFIRSTB", 1.08, 7500)
insert_index_component(constituents_weight_lots, "SBIN", 11.27, 750)
insert_index_component(constituents_weight_lots, "INDUSINDBK", 5.58, 500)
insert_index_component(constituents_weight_lots, "HDFCBANK", 27.04, 550)
insert_index_component(constituents_weight_lots, "KOTAKBANK", 11.72, 400)
insert_index_component(constituents_weight_lots, "BANDHANBNK", 1.98, 2800)
insert_index_component(constituents_weight_lots, "AXISBANK", 11.18, 625)

In [20]:
constituents_weight_lots

{'PNB': {'weight': 0.91, 'lot_size': 8000},
 'ICICIBANK': {'weight': 23.03, 'lot_size': 700},
 'AUBANK': {'weight': 2.69, 'lot_size': 1000},
 'BANKBARODA': {'weight': 1.84, 'lot_size': 2925},
 'FEDERALBNK': {'weight': 1.68, 'lot_size': 5000},
 'IDFCFIRSTB': {'weight': 1.08, 'lot_size': 7500},
 'SBIN': {'weight': 11.27, 'lot_size': 750},
 'INDUSINDBK': {'weight': 5.58, 'lot_size': 500},
 'HDFCBANK': {'weight': 27.04, 'lot_size': 550},
 'KOTAKBANK': {'weight': 11.72, 'lot_size': 400},
 'BANDHANBNK': {'weight': 1.98, 'lot_size': 2800},
 'AXISBANK': {'weight': 11.18, 'lot_size': 625}}

In [21]:
def normalize_constituents_weights(constituents_weight_lots):
    weight_sum = 0
    for values in constituents_weight_lots.values():
        weight_sum += values['weight']
    if weight_sum != 100:
        print(f"Sum of weights = {weight_sum} is not 100. Normalizing weights...")
        total_weight = sum([values['weight'] for values in constituents_weight_lots.values()])
        for values in constituents_weight_lots.values():
            values['weight'] = (values['weight'] / total_weight) * 100
    
        normalized_sum = sum([values['weight'] for values in constituents_weight_lots.values()])
        print("Normalized sum:", normalized_sum)
        print(constituents_weight_lots)
    else:
        print(f"provided weights add up to perfect {weight_sum}")
    return (constituents_weight_lots)

In [22]:
normalize_constituents_weights(constituents_weight_lots)

provided weights add up to perfect 100.0


{'PNB': {'weight': 0.91, 'lot_size': 8000},
 'ICICIBANK': {'weight': 23.03, 'lot_size': 700},
 'AUBANK': {'weight': 2.69, 'lot_size': 1000},
 'BANKBARODA': {'weight': 1.84, 'lot_size': 2925},
 'FEDERALBNK': {'weight': 1.68, 'lot_size': 5000},
 'IDFCFIRSTB': {'weight': 1.08, 'lot_size': 7500},
 'SBIN': {'weight': 11.27, 'lot_size': 750},
 'INDUSINDBK': {'weight': 5.58, 'lot_size': 500},
 'HDFCBANK': {'weight': 27.04, 'lot_size': 550},
 'KOTAKBANK': {'weight': 11.72, 'lot_size': 400},
 'BANDHANBNK': {'weight': 1.98, 'lot_size': 2800},
 'AXISBANK': {'weight': 11.18, 'lot_size': 625}}

In [23]:
constituents_weights_lots_data = {}

In [24]:
logs_fetching = open(f'LOGS_Fetching_Data_for_{strategy_desc}.txt', 'w')
sys.stdout = logs_fetching
for constituent_symbol, constituent in constituents_weight_lots.items():
    df_futures = data.get_futures_data_with_timestamps_between(constituent_symbol, expiry_type, start_date, end_date, timeframe)
    df_options = data.get_options_data_with_timestamps_between(constituent_symbol, expiry_type, start_date, end_date, timeframe)
    constituents_weights_lots_data[constituent_symbol] = constituents_weight_lots[constituent_symbol]
    constituents_weights_lots_data[constituent_symbol]['df_options'] = df_options
    constituents_weights_lots_data[constituent_symbol]['df_futures'] = df_futures
sys.stdout = original_stdout.value
logs_fetching.close()

In [25]:
logs_fetching = open(f'LOGS_Fetching_Data_for_{strategy_desc}.txt', 'a')
sys.stdout = logs_fetching
df_futures_index = data.get_futures_data_with_timestamps_between(index_symbol, expiry_type, start_date, end_date, timeframe)
df_options_index = data.get_options_data_with_timestamps_between(index_symbol, expiry_type, start_date, end_date, timeframe)
sys.stdout = original_stdout.value
logs_fetching.close()

In [26]:
constituents = {}

In [27]:
index = dp.ticker(index_symbol, df_futures_index, df_options_index, True, constituents, False, None, index_lot_size, timeframe, look_back_window, False, risk_free_rate)
for constituent_symbol, constituent in constituents_weights_lots_data.items():
    constituents[constituent_symbol] = dp.ticker(constituent_symbol, constituent['df_futures'], constituent['df_options'], False, None, True, constituent['weight']/100, constituent['lot_size'], timeframe, look_back_window, False, risk_free_rate)

In [28]:
ohlc = OHLC.close

In [29]:
index.set_ohlc(ohlc)
for _, constituent in constituents.items():
    constituent.set_ohlc(ohlc)

OHLC for BANKNIFTY is set to OHLC.close
OHLC for PNB is set to OHLC.close
OHLC for ICICIBANK is set to OHLC.close
OHLC for AUBANK is set to OHLC.close
OHLC for BANKBARODA is set to OHLC.close
OHLC for FEDERALBNK is set to OHLC.close
OHLC for IDFCFIRSTB is set to OHLC.close
OHLC for SBIN is set to OHLC.close
OHLC for INDUSINDBK is set to OHLC.close
OHLC for HDFCBANK is set to OHLC.close
OHLC for KOTAKBANK is set to OHLC.close
OHLC for BANDHANBNK is set to OHLC.close
OHLC for AXISBANK is set to OHLC.close


In [30]:
def get_index_and_components_iv(index):
    index.generate_iv_data()
    for _, component in index.components.items():
        component.generate_iv_data()

In [31]:
logs_iv = open(f'LOGS_IV_Generation_for_{strategy_desc}.txt', 'w')
sys.stdout = logs_iv
get_index_and_components_iv(index)
sys.stdout = original_stdout.value
logs_iv.close()

In [32]:
def get_pca_stocks(index, pct_variance_explained=90, total_stocks=5, n_stocks_for_each_principal_component=-1):
    data = pd.DataFrame(index=index.df_futures.index)
    
    # Create a DataFrame with IV data weighted by weight for each stock
    stock_symbols = []
    for comp in index.components.values():
        stock_symbols.append(comp.symbol)
        data[comp.symbol] = comp.df_futures['iv'] * comp.weight
        print(f"{comp.symbol} || weight: {round(comp.weight*100)}%")
    
    # Apply PCA
    pca = PCA(n_components=None)  # None means it will return all components
    pca.fit(data)

    # Get the explained variance ratio and components
    explained_variance = pca.explained_variance_ratio_
    cumulative_variance = np.cumsum(explained_variance)
    
    # Find the number of components that explain at least the required variance (e.g., 90%)
    k = np.argmax(cumulative_variance >= pct_variance_explained / 100) + 1
    
    print(f"Number of principal components selected: {k} || Variance explained: {round(cumulative_variance[k]*100)}%")
    print()
    
    # Get the top k components
    selected_components = pca.components_[:k]
    # To hold unique top stocks
    candidate_stocks = set()
    
    # Find the top contributing stocks for each selected component
    for i in range(k):
        component = selected_components[i]
        # Get the absolute values of the component to find the most contributing stocks
        stock_contributions = np.abs(component)
        # Sort the stocks by their contributions to the component
        top_stocks_idx = np.argsort(stock_contributions)[::-1]  # Indices of stocks sorted by contribution
        print(f"Variance Explained by Principal Component {i+1} = {round(explained_variance[i]*100)}% || Top contributing stocks:")
        for ii, idx in enumerate(top_stocks_idx):
            stock_symbol = stock_symbols[idx]
            string_added = ""
            if n_stocks_for_each_principal_component == -1 or ii < n_stocks_for_each_principal_component:
                candidate_stocks.add(idx)
                string_added = "(Candidate Stock, Top-contributing from this prinicpal component)" if n_stocks_for_each_principal_component != -1 else ""
            print(f"  {stock_symbol} - Contribution: {round(stock_contributions[idx], 4)} {string_added}")
        print()
    
    # sort stocks by their overall contributions and filter down to #top_stocks
    total_stock_contributions = np.zeros(len(stock_symbols))
    for idx in candidate_stocks:
        total_stock_contributions[idx] = np.sum(np.abs(selected_components[:, idx] * explained_variance[:k]))
    
    # Get top stocks overall
    overall_top_stocks_idx = np.argsort(total_stock_contributions)[::-1][:total_stocks]
    top_stocks = [stock_symbols[idx] for idx in overall_top_stocks_idx]

    print(f"Top {total_stocks} stocks based on PCA contributions:")
    for idx in overall_top_stocks_idx:
        stock = stock_symbols[idx]
        overall_contribution = total_stock_contributions[idx]
        print(f"  {stock} || original_weight = {round(index.components[stock].weight*100)}% || weighted importance = {round(overall_contribution)}")
    return top_stocks

In [33]:
pct_explained_pca = 90
total_stocks_to_pick = 5

In [34]:
logs_pca = open(f'LOGS_PCA_Dispersion_{strategy_desc}', 'w')
sys.stdout = logs_pca
pca_stocks = get_pca_stocks(index, pct_explained_pca, total_stocks_to_pick)
sys.stdout = original_stdout.value
logs_pca.close()

In [35]:
pca_stocks

['HDFCBANK', 'ICICIBANK', 'AXISBANK', 'KOTAKBANK', 'SBIN']

In [36]:
stocks_to_remove = [stock for stock, _ in index.components.items() if stock not in pca_stocks]
for stock in stocks_to_remove: 
    index.components.pop(stock)

In [37]:
weights_after_pca = {}
for component in index.components.values():
    insert_index_component(weights_after_pca, component.symbol, component.weight * 100, component.lot_size)
normalize_constituents_weights(weights_after_pca)
for symbol, values in weights_after_pca.items():
    index.components[symbol].weight = values['weight']/100
weights_after_pca

Sum of weights = 84.24000000000001 is not 100. Normalizing weights...
Normalized sum: 100.00000000000001
{'ICICIBANK': {'weight': 27.338556505223178, 'lot_size': 700}, 'SBIN': {'weight': 13.378442545109213, 'lot_size': 750}, 'HDFCBANK': {'weight': 32.098765432098766, 'lot_size': 550}, 'KOTAKBANK': {'weight': 13.912630579297247, 'lot_size': 400}, 'AXISBANK': {'weight': 13.271604938271606, 'lot_size': 625}}


{'ICICIBANK': {'weight': 27.338556505223178, 'lot_size': 700},
 'SBIN': {'weight': 13.378442545109213, 'lot_size': 750},
 'HDFCBANK': {'weight': 32.098765432098766, 'lot_size': 550},
 'KOTAKBANK': {'weight': 13.912630579297247, 'lot_size': 400},
 'AXISBANK': {'weight': 13.271604938271606, 'lot_size': 625}}

In [38]:
index.generate_ic_data()

Generating BANKNIFTY Implied Correlation(IC) data...
Implied correlation data for BANKNIFTY generated successfully


In [39]:
index.df_futures['zscore'] = (index.df_futures['ic'] - index.df_futures['ic'].rolling(window=index.look_back_window).mean())/index.df_futures['ic'].rolling(window=index.look_back_window).std()

ValueError: window must be an integer 0 or greater

In [38]:
print(index.df_futures['zscore'])

2023-07-03 09:15:00         NaN
2023-07-03 09:30:00         NaN
2023-07-03 09:45:00         NaN
2023-07-03 10:00:00         NaN
2023-07-03 10:15:00         NaN
                         ...   
2023-12-01 14:15:00    0.661758
2023-12-01 14:30:00    0.665588
2023-12-01 14:45:00    0.627355
2023-12-01 15:00:00    0.530391
2023-12-01 15:15:00    0.224568
Name: zscore, Length: 2600, dtype: float64


In [39]:
fig_ic = Plot.plot_df(index.df_futures, 'ic')
fig_ic.show()
Plot.save_plot(fig_ic, f'PLOT_ic_for_{strategy_desc}.html')

In [40]:
fig_zscore = Plot.plot_df(index.df_futures, 'zscore')
fig_zscore = Plot.draw_horizontal_line(fig_zscore, -2, 0, len(index.df_futures), 'Red')
fig_zscore = Plot.draw_horizontal_line(fig_zscore, zscore_threshold_short, 0, len(index.df_futures), 'Blue')
fig_zscore = Plot.draw_horizontal_line(fig_zscore, zscore_threshold_long, 0, len(index.df_futures), 'Blue')
fig_zscore = Plot.draw_horizontal_line(fig_zscore, 2, 0, len(index.df_futures), 'Red')
fig_zscore.show()
Plot.save_plot(fig_zscore, f'PLOT_zscore_for_{strategy_desc}.html')

In [41]:
x = index.df_futures['zscore']
count = 0
flag = False
for z in x:
    if z > zscore_threshold_long and flag == False:
        flag = True
        count+=1
    if z < zscore_threshold_short and flag == True:
        flag = False
        count += 1
print("theoretically number of trades just with zscore signals", count)

theoretically number of trades just with zscore signals 12


In [42]:
positions = {}

In [43]:
def get_straddle_vega(ticker, timestamp):
    vega = 0
    for option_type in Option:
        strike, _ = ticker.find_moneyness_strike(timestamp, 0, option_type)
        greeks = ticker.greeks(timestamp, option_type, strike)
        if greeks is None:
            print(f"get_straddle_vega: greeks is None || Symbol: {ticker.symbol} | OHLC: {ticker.ohlc.name} | Spot: {ticker.get_futures_price(timestamp)} | Strike: {strike} | Opt Price: {ticker.get_opts(option_type).loc[timestamp, strike]} | T(days): {pd.to_datetime(ticker.get_opts(option_type).loc[timestamp, 'expiry']).date() - pd.to_datetime(timestamp).date()} | Option: {option_type.name}")
            return None
        vega += greeks['vega']
    return vega

In [44]:
def get_lots_for_neutrality(v, notional_vega):
    lots_index = notional_vega/ v[0]
    lots_components = []
    for vega_component in v[1:]:
        lots_components.append(notional_vega/vega_component)
    return np.round(lots_index).astype(int), np.round(lots_components).astype(int)

In [45]:
def entry_signal(timestamp):
    zscore = index.df_futures.loc[timestamp, 'zscore']
    return zscore >= zscore_threshold_long


def exit_signal(timestamp):
    zscore = index.df_futures.loc[timestamp, 'zscore'] 
    return zscore <= zscore_threshold_short

def get_lots_for_dispersion(dispersion_position_of_index, index, timestamp, strike_notional = False, vega_neutral = False, theta_neutral = False, notional_vega = 100000):
    # If you want to do vega neutral entry
    if vega_neutral: 
        try:
            unweighted_vegas, weights, logging_information = {}, {}, {}
            vega_index_atm_straddle = get_straddle_vega(index, timestamp)
            if vega_index_atm_straddle is None:
                return None, None
            unweighted_vegas[index.symbol] = vega_index_atm_straddle
            weights[index.symbol] = 1
            logging_information[index.symbol] = {
                'Vega per Straddle': vega_index_atm_straddle,
                'Vega per Lot Straddle': vega_index_atm_straddle * index.lot_size,
                'Weight': index.weight,
                'Target Notional Vega': notional_vega
            }
            for _, component in index.components.items():
                vega_component_atm_straddle = get_straddle_vega(component, timestamp)
                if vega_component_atm_straddle is None:
                    return None, None
                unweighted_vegas[component.symbol] = vega_component_atm_straddle
                weights[component.symbol] = component.weight
                logging_information[index.symbol] = {
                    'Vega per Straddle': vega_component_atm_straddle,
                    'Vega per Lot Straddle': vega_component_atm_straddle * component.lot_size,
                    'Weight': component.weight,
                    'Target Notional Vega': notional_vega * component.weight
                }
            lots_index, lots_components = get_lots_for_neutrality(unweighted_vegas, weights, notional_vega, logging_information)
            for key, value in logging_information.items():
                information_string = " || ".join([f"{info_key} = {info_value}" for info_key, info_value in value.items()])
                output_string = f"{key} |||| {information_string}"
                print(output_string) 
            return lots_index, lots_components
        except Exception as e:
            print(f"get_lots_for_dispersion error at {timestamp}: {e}")
    # If you want to do a delta neutral entry
    if strike_notional:
        pass
    # If you want to do a theta/ correlation neutral entry
    if theta_neutral:
        pass

In [46]:
def get_net_delta(ticker, timestamp):
    net_delta = 0
    for _, token in ticker.tokens.items():
        net_delta += token.stats.loc[timestamp, 'net_delta']
    return net_delta

In [47]:
def take_dispersion_position(time_data, remarks, ticker, position, lots):
    timestamp, time_ix = time_data
    legs = []
    for option_type in Option:
        atm_strike, _ = ticker.find_moneyness_strike(timestamp, 0, option_type)
        key = f'{ticker.symbol}_{option_type.name}_{atm_strike}' 
        if key not in positions:
            positions[key] = TL.Token(ticker, option_type, atm_strike, FNO.OPTIONS)
        token = positions[key]
        token.add_position(lots, position)
        token.update_df(time_ix, timestamp)
        ticker.update_token(key, token)
        leg = {
            "position" : position, 
            "opt_type" : option_type, 
            "moneyness": 0, 
            "lots": lots
        }
        legs.append(leg)
    ticker.take_position(timestamp, remarks, *legs)


In [48]:
def check_existing_position(ticker):
    pos = False
    for _, token in ticker.tokens.items():
        pos = pos | (token.position.value != 0) 
    return pos

In [49]:
def hedge_delta(time_ix, timestamp, remarks, ticker, index_delta_threshold_per_lot = 5, component_delta_threshold_per_lot = 1, logging = True):
    net_delta = get_net_delta(ticker, timestamp)
    delta_threshold_per_lot = index_delta_threshold_per_lot if ticker.is_index else component_delta_threshold_per_lot 
    if abs(net_delta) >= delta_threshold_per_lot * ticker.lot_size:
        qty_futures = (-net_delta)/ticker.lot_size
        position = LongShort(np.sign(qty_futures))
        lots_futures = int(abs(qty_futures))
        if lots_futures != 0:
            ticker.hedge_futures_trade(lots_futures, position, timestamp)
            key = f'{ticker.symbol}_Futures'
            if key not in positions:
                positions[key] = TL.Token(ticker, None, None, FNO.FUTURES)
            token = positions[key]
            token.add_position(lots_futures, position)
            token.update_df(time_ix, timestamp, logging)
            ticker.update_token(key, token) #python obj ref prop, check
            token.ticker = ticker
            positions[key] = token #python obj ref prop, check
            leg = {
                "position" : position, 
                "opt_type" : FNO.FUTURES, 
                "moneyness": 0, 
                "lots": lots_futures
            }
            ticker.take_position(timestamp, remarks, leg)
    
            print(f">> HEDGED DELTA {ticker.symbol}: Added {lots_futures * position.value * ticker.lot_size} notional delta using {lots_futures} lots of future. new delta = {round(net_delta) + lots_futures * position.value * ticker.lot_size} || threshold per option = {delta_threshold_per_lot}")
        else:
            print(f">> Hedging condition threshold per option = {delta_threshold_per_lot} for {ticker.symbol} met but required lots_futures were 0 so did not hedge")
    else:
        print(f"XXXX DID NOT HEDGE  {ticker.symbol}'s {round(net_delta)} notional delta for {ticker.symbol} at {timestamp}. net_delta/{ticker.symbol}_lot_size_futures = {np.round(abs(net_delta)/ticker.lot_size)} < 1")

In [50]:
def reset_hedging_positions(ticker):
    ticker.hedging = pd.DataFrame(columns = ['date_timestamp', 'lots', 'position', 'futures_price', 'delta_added'])
    ticker.hedging.set_index('date_timestamp', inplace=True)

def reset_trade_positions(ticker):
    ticker.Trades = dp.Trades()
    ticker.tokens = {}

In [51]:
def HandleUpdate(ticker, time_ix, timestamp, remarks, hedge = True, logging_token_update=True, logging_hedging=True, index_delta_threshold_per_lot = 5, component_delta_threshold_per_lot = 1):
    for key, token in ticker.tokens.items():
        token.update_df(time_ix, timestamp, logging_token_update)
        positions[key] = token #python obj ref prop, check
        if not logging_hedging:
            continue
        if token.fno == FNO.OPTIONS:
            print(f"{token.ticker.symbol, token.opt_type.name, float(token.strike/100)} | Delta per lot = {np.round(token.stats.loc[timestamp, 'net_delta'], 2)} (position accounted) | Notional Delta = {np.round(token.stats.loc[timestamp, 'net_delta'], 2) * ticker.lot_size} | Lot size: {ticker.lot_size} | Lots in position: {token.lots} | position = {token.position.name}")
        else:
            print(f"{token.ticker.symbol, token.fno.name} | Delta per lot = {np.round(token.stats.loc[timestamp, 'net_delta'], 2)} | Notional Delta = {np.round(token.stats.loc[timestamp, 'net_delta'], 2) * ticker.lot_size} | Lot size: {ticker.lot_size} | Lots in position: {token.lots} | position = {token.position.name}")
    if hedge:
        hedge_delta(time_ix, timestamp, remarks, ticker, index_delta_threshold_per_lot, component_delta_threshold_per_lot)
    for key, token in ticker.tokens.items():
        token.ticker = ticker
        positions[key] = token #python obj ref prop, check

In [52]:
def squareoff_ticker(time_ix, timestamp, remarks, ticker, logging):
    legs_trade, leg_hedge = [], []
    for key, token in ticker.tokens.items():
        if token.position == LongShort.Neutral:
            continue
        position_to_take = LongShort(-token.position.value) 
        if token.fno == FNO.OPTIONS:
            leg = {
            "position" : position_to_take, 
            "opt_type" : token.opt_type, 
            "moneyness": 0, 
            "lots": token.lots
            }
            legs_trade.append(leg)
        else:
            leg = {
            "position" : position_to_take, 
            "opt_type" : FNO.FUTURES, 
            "moneyness": 0, 
            "lots": token.lots
            }
            leg_hedge.append(leg)
        token.add_position(token.lots, position_to_take)
        token.update_df(time_ix, timestamp, logging)
        positions[key] = token #python obj ref prop, check
        ticker.tokens[key] = token #python obj ref prop, check
    if len(legs_trade):
        ticker.take_position(timestamp, remarks, *legs_trade)
    else:    
        print(f"?? legs_trade is empty for {ticker.symbol}")
    if len(leg_hedge):
        ticker.take_position(timestamp, remarks, *leg_hedge)
    else:
        print(f"?? leg_hedge is empty for {ticker.symbol}")
        
    # Every Token in a Ticker should have the updated Ticker in them
    for key, token in ticker.tokens.items():
        token.ticker = ticker #python obj ref prop, check
        positions[key] = token #python obj ref prop, check

In [53]:
def squareoff(time_ix, timestamp, remarks, logging = True):
    print("************  SQUARE OFF BEGINS  *********** ")
    # for key, token in positions.items():
    #     token.add_position(token.lots, LongShort(-token.position.value))
    # print("******  positions added in GLOBAL TOKENS. Now updating the key-information(running_pnl, lots, !(delta, vega, iv)) in them  ******")
    # for key, token in positions.items():
    #     token.update_df(time_ix, timestamp, logging)
    #     positions[key] = token
    # print("******  Information updated globally. TICKER WISE UPDATION BEGINS  ******")
    squareoff_ticker(time_ix, timestamp, remarks, index, logging)
    for component in index.components.values():
        squareoff_ticker(time_ix, timestamp, remarks, component, logging)
    print("* TRADE SAVED, TOKENS in ticker UPDATED")
    print("************  SQUARE OFF COMPLETE  *********** ")


In [54]:
class DispersionPTSL(TL.PTSLHandling):
    def __init__(self, profit_target, stop_loss, *tickers):
        super().__init__(profit_target, stop_loss, *tickers)

    def is_valid(self, timestamp):
        if not self.flag:
            return True
        index = self.tickers[0]
        for z in index.df_futures['zscore'].loc[self.triggered_at : timestamp]:
            if abs(z) <= epsilon:
                return True
        return False   

In [55]:
def isTodayAnyExpiry(timestamp, index):
    is_expiry = False
    is_expiry = is_expiry or (pd.to_datetime(timestamp).date() == pd.to_datetime(index.get_opts(Option.Call).loc[timestamp, 'expiry']).date())
    for component in index.components.values():
        is_expiry = is_expiry or (pd.to_datetime(timestamp).date() == pd.to_datetime(component.get_opts(Option.Call).loc[timestamp, 'expiry']).date())
    return is_expiry

In [56]:
positions = {}
reset_hedging_positions(index)
reset_trade_positions(index)
for component in index.components.values():
    reset_hedging_positions(component)
    reset_trade_positions(component)


PTSL = DispersionPTSL(profit_target, stop_loss, index, *index.components.values())

file = open(f'LOGS_Trading_Logic_for_{strategy_desc}.txt', 'w')
sys.stdout = file

for time_ix, timestamp in enumerate(index.df_futures.index):
    print()
    print(f"{timestamp}")
    #######################################################################################################################################################
    if check_existing_position(index):
        
        print("Existing Position check: TRUE")
        
        # EXPIRY DAY MANUAL SQUAREOFF
        if isTodayAnyExpiry(timestamp, index):
            print("************  EXPIRY DAY, MANUAL SQUARE OFF ************  ")
            squareoff(time_ix, timestamp, 'Expiry Day, Manual Square Off', False)
            HandleUpdate(index, time_ix, timestamp, 'NoHedging', False, False, False, index_delta_threshold_per_lot, component_delta_threshold_per_lot)
            for component in index.components.values():
                HandleUpdate(component, time_ix, timestamp, 'NoHedging', False, False, False, index_delta_threshold_per_lot, component_delta_threshold_per_lot)
            continue
            
            
        # PROFIT TARGET AND STOP LOSS SQUAREOFF
        ptsl_result = PTSL.status(timestamp) 
        if ptsl_result != "good_to_go":
            print("************  PTSL Trigger hit *********** ")
            squareoff(time_ix, timestamp, f'{ptsl_result} Square Off', True)
            HandleUpdate(index, time_ix, timestamp, 'NoHedging', False, True, False, index_delta_threshold_per_lot, component_delta_threshold_per_lot)
            for component in index.components.values():
                HandleUpdate(component, time_ix, timestamp, 'NoHedging', False, True, False, index_delta_threshold_per_lot, component_delta_threshold_per_lot)
            continue
            
            
        # REVERSE SIGNAL SQUAREOFF
        squareoff_signal = exit_signal(timestamp) if flag else entry_signal(timestamp)
        if squareoff_signal:
            print("Entered squareoff signal block")
            print("************  SquareOff Reverse zscore Signal hit. ************ ")
            squareoff(time_ix, timestamp, 'ZScore Reversal Square Off (Entry in next timestamp/ signal)')
            HandleUpdate(index, time_ix, timestamp, 'NoHedging', False, True, False, index_delta_threshold_per_lot, component_delta_threshold_per_lot)
            for component in index.components.values():
                HandleUpdate(component, time_ix, timestamp, 'NoHedging', False, True, False, index_delta_threshold_per_lot, component_delta_threshold_per_lot)
            continue
            
        # IF NO NEED TO SQUAREOFF, HEDGE IF NEEDED
        HandleUpdate(index, time_ix, timestamp, 'Hedge', True, True, True, index_delta_threshold_per_lot, component_delta_threshold_per_lot)
        for component in index.components.values():
            HandleUpdate(component, time_ix, timestamp, 'Hedge', True, True, True, index_delta_threshold_per_lot, component_delta_threshold_per_lot)
        print("================")
        continue
    #######################################################################################################################################################
    
    
    #######################################################################################################################################################
    if entry_signal(timestamp):
        print("************  ENTRY SIGNAL ************  ")
        if isTodayAnyExpiry(timestamp, index):
            print("Expiry Day today. Trading on Expiry Day is not Allowed")
        
        elif not PTSL.is_valid(timestamp):
            print(f"PTSL Triggered at {PTSL.triggered_at} and has not yet reverted to mean")
        
        else:
            try:
                lots_index, lots_components = get_lots_for_dispersion(LongShort.Short, index, timestamp, False, True, False, notional_vega)
                print(f"lots_index, lots_components: {lots_index, lots_components}")
    
                take_dispersion_position((timestamp, time_ix), 'Long Dispersion Signal (zscore high, short index, long components)', index, LongShort.Short, lots_index)
                for (component, lots_component) in zip(index.components.values(), lots_components):
                    take_dispersion_position((timestamp, time_ix), 'Long Dispersion Signal (zscore high, short index, long components)', component, LongShort.Long, lots_component)
                PTSL.fresh_trade(timestamp)
                print(f"Long dispersion (Short Index Long Components) trade when zscore was {index.df_futures.loc[timestamp, 'zscore']}")
            except Exception as e:
                print(f"Can't take trade due to Non availability of greeks -> cant enter a vega neutral position")
                print(f"take_dispersion_position error at {timestamp}: {e}")  
 
    elif exit_signal(timestamp):
        print("************  EXIT SIGNAL ************  ")
        if isTodayAnyExpiry(timestamp, index):
            print("Expiry Day today. Trading on Expiry Day is not Allowed")
        
        elif not PTSL.is_valid(timestamp):
            print(f"PTSL Triggered at {PTSL.triggered_at} and has not yet reverted to mean")
        
        else:
            try:
                lots_index, lots_components = get_lots_for_dispersion(LongShort.Long, index, timestamp, False, True, False, notional_vega)
                print(f"lots_index, lots_components: {lots_index, lots_components}")
    
                take_dispersion_position((timestamp, time_ix), 'Short Dispersion Signal (zscore low, long index, short components)', index, LongShort.Long, lots_index)
                for (component, lots_component) in zip(index.components.values(), lots_components):
                    take_dispersion_position((timestamp, time_ix), 'Short Dispersion Signal (zscore low, long index, short components)', component, LongShort.Short, lots_component)
                PTSL.fresh_trade(timestamp)
                print(f"Short dispersion (Long Index Short Components) trade when zscore was {index.df_futures.loc[timestamp, 'zscore']}")
            except Exception as e:
                print(f"Can't take trade due to Non availability of greeks -> cant enter a vega neutral position")
                print(f"take_dispersion_position error at {timestamp}: {e}") 
    #######################################################################################################################################################


    HandleUpdate(index, time_ix, timestamp, 'NoHedging', 
                 hedge=False, logging_token_update=True, logging_hedging=False, 
                 index_delta_threshold_per_lot=index_delta_threshold_per_lot, 
                 component_delta_threshold_per_lot=component_delta_threshold_per_lot)
    for component in index.components.values():
        HandleUpdate(component, time_ix, timestamp, 'NoHedging', 
                     hedge=False, logging_token_update=True, logging_hedging=False, 
                     index_delta_threshold_per_lot=index_delta_threshold_per_lot, 
                     component_delta_threshold_per_lot=component_delta_threshold_per_lot)

file.close()
sys.stdout = original_stdout.value

In [57]:
data = {
    'running_pnl': [],
    'net_delta': [],
    'net_vega': [],
    'net_lots': [],
    'active_position': []
}

for timestamp in index.df_futures.index:
    pnl = 0
    delta = 0
    vega = 0
    lots = 0

    for _, token in positions.items():
        stats_at_timestamp = token.stats.loc[timestamp]
        pnl += stats_at_timestamp['running_pnl']
        delta += stats_at_timestamp['net_delta']
        vega += stats_at_timestamp['net_vega']
        lots += stats_at_timestamp['net_lots']

    active_position = int(abs(lots) > 0)
    
    data['running_pnl'].append(pnl) 
    data['net_delta'].append(delta)
    data['net_vega'].append(vega)
    data['net_lots'].append(lots)
    data['active_position'].append(active_position)
result_df = pd.DataFrame(data, index=index.df_futures.index)

In [58]:
result_df['active_position'] *= 500

In [59]:
fig_pnl = Plot.plot_df(result_df, 'running_pnl')
Plot.save_plot(fig_pnl, f'PLOT_PNL_for_{strategy_desc}.html')

In [60]:
fig_greeks = Plot.plot_df(result_df, 'net_delta', 'net_vega')
Plot.save_plot(fig_greeks, f'PLOT_GREEKS_for_{strategy_desc}.html')

In [61]:
fig_signals = Plot.plot_df(index.df_futures, 'zscore', 'ic')
fig_signals = Plot.draw_horizontal_line(fig_signals, zscore_threshold_short, index.df_futures['ix'].iloc[0], index.df_futures['ix'].iloc[-1], 'white')
fig_signals = Plot.draw_horizontal_line(fig_signals, zscore_threshold_long, index.df_futures['ix'].iloc[0], index.df_futures['ix'].iloc[-1], 'white')
Plot.save_plot(fig_signals, f'PLOT_SIGNALS_for_{strategy_desc}.html')

In [62]:
arr = []
arr += [{**trade, 'symbol': index.symbol} for trade in index.Trades.tradesArr]
for comp in index.components.values():
    arr += [{**trade, 'symbol': comp.symbol} for trade in comp.Trades.tradesArr]

In [63]:
df_trades = pd.DataFrame(arr)
df_trades = df_trades.sort_values(by='timestamp')
df_trades.reset_index(inplace=True)
df_trades.drop(columns=['index'], inplace=True)

KeyError: 'timestamp'

In [65]:
df_trades.to_excel(f"INFO_Trades_for_{strategy_desc}.xlsx", index=False)
Plot.save_excel_trades(f"INFO_Trades_for_{strategy_desc}.xlsx")

In [66]:
def metrics(fund_blocked):
    info_metrics = open(f'INFO_Metrics_for_{strategy_desc}.txt', 'w')
    sys.stdout = info_metrics

    print(f"TimeFrame Length: 6 months")
    print(f"Fundblocked in rs: {fund_blocked/100}")
    result_df.index = pd.to_datetime(result_df.index)
    result_df['date'] = result_df.index.date  # Create a new column for the date
    daily_profit = result_df.groupby('date')['running_pnl'].agg(lambda x: x.iloc[-1] - x.iloc[0])
    daily_profit.index = pd.to_datetime(result_df['date'].unique())
    
    ret = daily_profit.sum()/fund_blocked * 100
    print(f"Return: {round(ret)}%")
    
    sharpe_value = (daily_profit.mean() - risk_free_rate*fund_blocked/365)/daily_profit.std()
    print(f"Sharpe: {round(sharpe_value)}")
    
    peaks = result_df['running_pnl'].cummax()
    drawdowns = (result_df['running_pnl'] - peaks)/(peaks)
    max_drawdown = drawdowns.min()
    print(f"Drawdown: {round(max_drawdown)}%")
    monthly_data = daily_profit.resample('ME').sum()
    monthly_profit = monthly_data[monthly_data > 0].sum()
    monthly_loss = abs(monthly_data[monthly_data < 0].sum())
    if monthly_loss != 0:
        profit_loss_ratio = monthly_profit / monthly_loss
    else:
        profit_loss_ratio = float('inf')  # Avoid division by zero
    print("Monthly Profit/Loss Ratio:", round(profit_loss_ratio))
    
    windays = (result_df['running_pnl'] > 0).sum()
    lossdays = (result_df['running_pnl'] < 0).sum()
    if (windays + lossdays) > 0:
        win_ratio = windays / (windays + lossdays)
    else:
        win_ratio = 0
    print("Win Ratio:", round(win_ratio))
    
    streak = (result_df['running_pnl'] > 0).astype(int)  # 1 for winning days, 0 otherwise
    losing_streak = (result_df['running_pnl'] < 0).astype(int)  # 1 for losing days, 0 otherwise
    winning_streaks = streak.groupby((streak != streak.shift()).cumsum()).cumsum()
    losing_streaks = losing_streak.groupby((losing_streak != losing_streak.shift()).cumsum()).cumsum()
    print(f"Longest Winning Streak: {winning_streaks.max()}, Longest Losing Streak: {losing_streaks.max()}")

    sys.stdout = original_stdout.value
    info_metrics.close()

In [67]:
metrics(100000)